In [5]:
from fastai.tabular.all import *
import pandas as pd
import numpy as np
import pickle

In [6]:
with open(f"cache/training_data.pickle", 'rb') as f:
    data = pickle.load(f)
columns_output = list(map(str, list(data['limits'])))
degrees = data['degrees']
lengths = data['lengths']
degree_length_inputs = []
for degree in degrees:
    for length in lengths:
        degree_length_inputs.append(f"{degree}-{length}")

In [7]:
input_symbols = np.array(data['input_symbols'])
input_symbols_unique = set(data['input_symbols'])
data_input = pd.DataFrame(data=data['input'].transpose(), columns=degree_length_inputs)
#data_input['symbol'] = data['input_symbols']
#data_input['symbol'] = data_input['symbol'].astype('|S')
for symbol in input_symbols_unique:
    data_input[symbol] = np.where(input_symbols == symbol, True, False)

In [9]:
len(input_symbols_unique), input_symbols_unique

(19,
 {'ADAUSDT',
  'BCHUSDT',
  'BNBUSDT',
  'BTCUSDT',
  'BTTUSDT',
  'CHZUSDT',
  'DOGEUSDT',
  'EOSUSDT',
  'ETCUSDT',
  'ETHUSDT',
  'LINKUSDT',
  'LTCUSDT',
  'MATICUSDT',
  'NEOUSDT',
  'THETAUSDT',
  'TRXUSDT',
  'VETUSDT',
  'XLMUSDT',
  'XRPUSDT'})

In [10]:
data_output = pd.DataFrame(data=data['output'].transpose(), columns=columns_output)

In [11]:
df = pd.concat((data_input, data_output.reindex(data_input.index)), axis=1)
df.columns = list(map(str, list(df.columns)))
del data_input
del data_output

In [12]:
cont_names = degree_length_inputs
cat_names = list(input_symbols_unique)
y_names = list(map(str, list(data['limits'])))

In [13]:
splits = RandomSplitter(valid_pct=0.2)(range_of(df))

In [14]:
to = TabularPandas(df, procs=[Categorify], cat_names=cat_names, cont_names=cont_names, y_names=y_names, splits=splits)

In [15]:
dls = to.dataloaders(bs=2**17)

In [16]:
learn = tabular_learner(dls, metrics=rmse)

In [17]:
learn.fit_one_cycle(50)

epoch,train_loss,valid_loss,_rmse,time
0,0.675583,0.532083,0.729440,01:12
1,0.597844,0.557138,0.746417,01:10
2,0.546086,0.507523,0.712407,01:11
3,0.476087,0.410305,0.640550,01:13
4,0.353074,0.264514,0.514309,01:10
5,0.262082,0.231359,0.480998,01:10
6,0.237035,0.229429,0.478987,01:11
7,0.230493,0.228373,0.477884,01:12
8,0.228179,0.227079,0.476528,01:11
9,0.226733,0.225903,0.475292,01:10


In [18]:
learn.show_results(max_n=50)

,XLMUSDT,ADAUSDT,EOSUSDT,VETUSDT,NEOUSDT,DOGEUSDT,MATICUSDT,THETAUSDT,LTCUSDT,BTCUSDT,XRPUSDT,BTTUSDT,BNBUSDT,ETHUSDT,BCHUSDT,ETCUSDT,TRXUSDT,CHZUSDT,LINKUSDT,1-5,1-7,1-11,1-15,1-22,1-33,1-47,1-68,1-100,2-5,2-7,2-11,2-15,2-22,2-33,2-47,2-68,2-100,3-5,3-7,3-11,3-15,3-22,3-33,3-47,3-68,3-100,"(1.03, 0.93)","(1.04, 0.94)","(1.05, 0.95)","(1.06, 0.96)","(1.07, 0.97)","(1.03, 0.93)_pred","(1.04, 0.94)_pred","(1.05, 0.95)_pred","(1.06, 0.96)_pred","(1.07, 0.97)_pred"
0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-0.003145,-0.002005,0.001035,0.000864,0.001205,0.000400,-0.000297,0.000196,-0.000038,-0.010301,-0.007462,-0.004020,-0.000211,0.000021,0.002781,0.001803,-0.000772,0.000482,-0.010172,-0.008615,-0.010766,-0.006986,-0.001159,-0.001841,0.003086,0.002715,-0.000817,1.0,1.0,0.0,0.0,0.0,0.574319,0.479929,0.383054,0.295827,0.242548
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.003613,0.000348,-0.001000,-0.000859,-0.000433,0.000898,0.000766,0.000194,-0.000057,0.003095,0.006694,0.001358,-0.000095,-0.002139,-0.002277,0.000082,0.001633,0.000669,0.004251,0.003634,0.008542,0.002565,0.002125,-0.002649,-0.002182,-0.000734,0.002190,1.0,1.0,1.0,1.0,1.0,0.758611,0.710880,0.624853,0.558424,0.460195
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-0.000138,0.000037,-0.001707,-0.001526,-0.000665,-0.000211,-0.000419,-0.000228,-0.000063,-0.001661,-0.000142,0.002429,-0.001312,-0.002343,-0.001330,0.000128,-0.000940,-0.000322,-0.003978,-0.003336,-0.001223,0.003975,-0.001579,-0.003281,-0.002655,0.000823,-0.001671,1.0,1.0,1.0,1.0,1.0,0.807561,0.745150,0.649867,0.534029,0.416317
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-0.001434,0.000920,0.001112,-0.000244,-0.002023,-0.001034,-0.000954,-0.001356,-0.000530,-0.004133,-0.005096,0.002079,0.004116,0.002009,-0.002682,-0.001540,-0.000364,-0.002167,-0.000770,-0.002677,-0.007489,-0.002212,0.006325,0.002842,-0.002148,-0.001847,-0.001146,0.0,0.0,0.0,0.0,0.0,0.713965,0.699674,0.589240,0.465466,0.386623
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-0.002088,0.000088,0.001333,0.001750,0.002419,0.001810,0.001051,0.000989,-0.000044,-0.005835,-0.005015,-0.000373,-0.000025,0.000424,0.003102,0.003181,0.001277,0.002726,0.000500,-0.005313,-0.007491,-0.001855,-0.000539,-0.000176,0.003321,0.004752,0.001502,1.0,1.0,1.0,1.0,0.0,0.908663,0.795417,0.766831,0.722054,0.579027
5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0.001827,0.000617,-0.000173,-0.000549,-0.000494,-0.000903,-0.000454,-0.000147,0.000290,-0.001128,0.002131,0.001503,0.001046,-0.000303,0.000427,-0.000993,-0.000970,-0.000898,-0.004122,-0.001723,0.002501,0.001630,0.001668,-0.000071,-0.000118,-0.001069,-0.001271,1.0,0.0,0.0,0.0,0.0,0.725814,0.683173,0.622120,0.569904,0.475824
6,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-0.000547,-0.000646,0.001510,0.002214,0.002478,0.002444,0.002046,0.001055,0.000314,0.004269,0.001269,-0.002749,-0.000169,0.001332,0.002344,0.003272,0.003920,0.002600,0.008634,0.004823,0.001610,-0.004070,-0.001400,0.000629,0.001184,0.002828,0.004770,1.0,1.0,1.0,1.0,1.0,0.949785,0.862345,0.769335,0.754097,0.622623
7,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,-0.001261,-0.000575,0.000649,-0.000260,-0.001003,-0.000131,0.000032,0.000653,0.000517,-0.003171,-0.001201,-0.000492,0.001541,0.000700,-0.001513,-0.000746,-0.001110,0.001033,-0.001459,-0.005230,-0.004918,-0.001611,0.002317,-0.000106,-0.001455,-0.001257,-0.002920,1.0,0.0,0.0,0.0,0.0,0.686993,0.577737,0.495246,0.359712,0.256331
8,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-0.003852,-0.003670,-0.003036,-0.002586,-0.002660,-0.001973,-0.001458,-0.000730,-0.000192,0.003115,-0.002129,-0.003780,-0.003081,-0.002101,-0.004294,-0.003201,-0.003011,-0.002312,0.007516,0.004520,-0.001136,-0.005461,-0.004590,-0.000470,-0.004503,

In [19]:
learn.export('model_all.pickle')